Team *Spatial Extent*

**Ensamble Model**

This is an ensamble of the fine tuned VGG, Resnet and EfficientNet. The idea was to improve the performances of the best one (VGG), but it has not been able to do so (even if the score is quiet good), probably because the other two models are too weak wrt to the first

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json #read json labels file
from PIL import Image #images operations
from datetime import datetime
from sklearn.utils import shuffle 
import math


# Get current working directory
cwd = os.getcwd()

In [ ]:
SEED=1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/_Polimi/artificial-neural-networks-and-deep-learning-2020.zip'

Output streaming troncato alle ultime 5000 righe.
  inflating: MaskDataset/training/10891.jpg  
  inflating: MaskDataset/training/10892.jpg  
  inflating: MaskDataset/training/10894.jpg  
  inflating: MaskDataset/training/10899.jpg  
  inflating: MaskDataset/training/10901.jpg  
  inflating: MaskDataset/training/10902.jpg  
  inflating: MaskDataset/training/10905.jpg  
  inflating: MaskDataset/training/10906.jpg  
  inflating: MaskDataset/training/10907.jpg  
  inflating: MaskDataset/training/10908.jpg  
  inflating: MaskDataset/training/10911.jpg  
  inflating: MaskDataset/training/10913.jpg  
  inflating: MaskDataset/training/10915.jpg  
  inflating: MaskDataset/training/10916.jpg  
  inflating: MaskDataset/training/10917.jpg  
  inflating: MaskDataset/training/10919.jpg  
  inflating: MaskDataset/training/10920.jpg  
  inflating: MaskDataset/training/10921.jpg  
  inflating: MaskDataset/training/10922.jpg  
  inflating: MaskDataset/training/10923.jpg  
  inflating: MaskDataset/train

In [ ]:
dataset_dir = os.path.join(os.getcwd(), 'MaskDataset')
training_dir = os.path.join(dataset_dir,'training')

In [ ]:
img_w = 612
img_h = 408
num_classes = 3

# VGG 16

In [ ]:
vgg_base = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

vgg = tf.keras.Sequential()
vgg.add(vgg_base)
vgg.add(tf.keras.layers.Flatten())
vgg.add(tf.keras.layers.Dense(units=512, activation='relu'))
vgg.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

58892288/58889256 [==============================] - 1s 0us/step


Link VGG weights: https://drive.google.com/drive/folders/1-kYwRSl01C0fD0XVzEZiuWVqQMZ0FViI?usp=sharing

In [ ]:
ckpt_dir = '/content/drive/My Drive/_Polimi/NN/Modello da 0.9533/cp_01.ckpt'
vgg.load_weights(ckpt_dir)

# ResNet v2

In [ ]:
resnet_base = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

resnet = tf.keras.Sequential()
resnet.add(resnet_base)
resnet.add(tf.keras.layers.Flatten())
resnet.add(tf.keras.layers.Dense(units=512, activation='relu'))
resnet.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

94674944/94668760 [==============================] - 1s 0us/step


Link ResNet weights: https://drive.google.com/drive/folders/12KqCXhW9k_qd3tctO1uMAJcyf4TwQ_Am?usp=sharing

In [ ]:
ckpt_dir = '/content/drive/My Drive/_Polimi/NN/transfer_learning_experiments_ResNet/CNN_Nov19_11-42-49/ckpts/cp_11.ckpt'
resnet.load_weights(ckpt_dir)

# EfficientNet B0

In [ ]:
base_efficient = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))


efficient = tf.keras.Sequential()
efficient.add(base_efficient)
efficient.add(tf.keras.layers.Flatten())
efficient.add(tf.keras.layers.Dense(units=512, activation='relu'))
efficient.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

16711680/16705208 [==============================] - 0s 0us/step


Link EfficientNet weights https://drive.google.com/drive/folders/1321AbjXAETP-6rRKF1q1xF1VgJB5eDSa?usp=sharing

In [ ]:
ckpt_dir = '/content/drive/My Drive/_Polimi/NN/transfer_learning_experiments_EfficientNet/CNN_Nov20_17-01-19/ckpts/cp_07.ckpt'
efficient.load_weights(ckpt_dir)

# Prediction

In [ ]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

**1st Voting Mechanism**: majority voting, with precedence to VGG (the best one) in the (rare) case of tie

In [ ]:
from collections import Counter

image_filenames = next(os.walk('MaskDataset/test'))[2]


results = {}
for image_name in image_filenames:
    img = Image.open('/content/MaskDataset/test/' + image_name).convert('RGB')
    img = img.resize((img_w, img_h))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, 0)

    votes = []

    img_array_vgg = tf.keras.applications.vgg16.preprocess_input(img_array)
    softmax_vgg = vgg.predict(img_array_vgg)
    prediction_vgg = np.argmax(softmax_vgg) 
    votes.append(prediction_vgg)

    img_array_resnet = tf.keras.applications.resnet_v2.preprocess_input(img_array)
    softmax_resnet = resnet.predict(img_array_resnet)
    prediction_resnet = np.argmax(softmax_resnet)
    votes.append(prediction_resnet)

    img_array_eff = img_array
    softmax_eff = efficient.predict(img_array_eff)
    prediction_eff = np.argmax(softmax_eff) 
    votes.append(prediction_eff)

    c = Counter(votes)
    res = c.most_common(1)[0] #returns a tuple with (class, count) where class is the most-voted class
    if res[1] > 1:
      prediction = res[0]
    else:
      prediction = prediction_vgg #prefer vgg in case of tie
    print(str(prediction_vgg) + ", " + str(prediction_resnet) + ", " + str(prediction_eff) + " = " + str(prediction))
    results[image_name] = prediction

create_csv(results)

0, 0, 0 = 0
2, 2, 2 = 2
1, 1, 1 = 1
0, 0, 0 = 0
0, 0, 0 = 0
0, 0, 0 = 0
0, 0, 0 = 0
0, 0, 0 = 0
1, 2, 1 = 1
2, 2, 2 = 2
1, 1, 1 = 1
0, 0, 0 = 0
1, 1, 1 = 1
0, 0, 0 = 0
0, 0, 0 = 0
0, 0, 0 = 0
2, 1, 2 = 2
0, 0, 0 = 0
2, 2, 2 = 2
0, 0, 0 = 0
1, 1, 1 = 1
0, 0, 0 = 0
0, 0, 0 = 0
1, 1, 1 = 1
2, 2, 2 = 2
2, 2, 2 = 2
2, 1, 2 = 2
2, 1, 1 = 1
2, 2, 2 = 2
0, 0, 0 = 0
1, 1, 2 = 1
0, 2, 0 = 0
0, 0, 0 = 0
0, 0, 0 = 0
0, 0, 0 = 0
0, 0, 0 = 0
1, 1, 1 = 1
2, 2, 2 = 2
2, 1, 1 = 1
1, 1, 2 = 1
1, 1, 1 = 1
0, 0, 0 = 0
2, 2, 2 = 2
0, 0, 0 = 0
0, 0, 0 = 0
1, 1, 1 = 1
1, 1, 1 = 1
2, 2, 2 = 2
1, 2, 1 = 1
2, 2, 2 = 2
0, 0, 0 = 0
1, 1, 1 = 1
0, 0, 0 = 0
1, 1, 1 = 1
2, 2, 2 = 2
2, 2, 1 = 2
1, 1, 1 = 1
2, 2, 2 = 2
2, 2, 2 = 2
2, 2, 2 = 2
1, 1, 1 = 1
2, 2, 2 = 2
1, 1, 1 = 1
1, 1, 1 = 1
2, 2, 2 = 2
1, 1, 2 = 1
1, 1, 1 = 1
0, 0, 0 = 0
2, 2, 2 = 2
2, 2, 2 = 2
2, 2, 2 = 2
0, 0, 0 = 0
2, 1, 2 = 2
2, 1, 0 = 2
2, 1, 1 = 1
0, 0, 0 = 0
2, 2, 2 = 2
0, 0, 0 = 0
2, 2, 1 = 2
1, 1, 1 = 1
1, 1, 1 = 1
0, 0, 0 = 0
1, 1, 1 = 1
2, 2

**2nd voting mechanism:** sum the scores of the models and then argmax

In [ ]:
from collections import Counter

image_filenames = next(os.walk('MaskDataset/test'))[2]


results = {}
for image_name in image_filenames:
    img = Image.open('/content/MaskDataset/test/' + image_name).convert('RGB')
    img = img.resize((img_w, img_h))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, 0)

    img_array_vgg = tf.keras.applications.vgg16.preprocess_input(img_array)
    softmax_vgg = vgg.predict(img_array_vgg)

    img_array_resnet = tf.keras.applications.resnet_v2.preprocess_input(img_array)
    softmax_resnet = resnet.predict(img_array_resnet)

    img_array_eff = img_array
    softmax_eff = efficient.predict(img_array_eff)

    softmax = softmax_eff + softmax_resnet + softmax_vgg

    prediction = np.argmax(softmax) 


    #print(str(softmax_vgg) + ", " + str(softmax_resnet) + ", " + str(softmax_eff) + " = " + str(softmax) + " => " + str(prediction))
    results[image_name] = prediction

create_csv(results)